In [1]:
import math
import os, sys
import shutil
import time
from datetime import timedelta

import numpy as np
import pandas as pd

import geopandas as gpd
from shapely.geometry import Point #, Polygon, mapping
# from shapely.geometry.polygon import orient
from statistics import mean
import h5py
# # To read KML files with geopandas, we will need to enable KML support in fiona (disabled by default)
# fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'
import json
import zipfile
import io

import requests

from xml.etree import ElementTree as ET
import logging
# from rema_dem import get_rema_strip_polygon
from create_dem_metadata import get_strip_polygon

# import warnings  # March 30, 2022  
# warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
# warnings.simplefilter(action='ignore', category=UserWarning) # UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
# # One more: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.

# Some global variables and definition [must choose one]
short_name = 'ATL06'  # 'ATL08' 'ATL06' 'ATL03'


In [2]:
region = 'arcticdem_02_greenland_southeast' #args.region  # sys.argv[1] #pass region from outside
dem_type = 'ArcticDEM' #args.dem_type # dem_type = ['REMA', 'ArcticDEM', 'EarthDEM']

logging.basicConfig(filename=f'logs_{region}.log', level=logging.INFO, format='%(asctime)s:%(levelname)s:%(message)s')
# Location of where DEMs are staged   
# region = arcticdem_05_greenland_northeast arcticdem_06_greenland_northwest arcticdem_05_greenland_northeast arcticdem_04_greenland_central arcticdem_05_greenland_northeast  
# region = rema_01_subantarctic_islands region_03_peninsula_south hma_2019jun26 region_20_ross_shelf
if dem_type == 'ArcticDEM':
    dir_prefix = '/fs/byo/howat-data5'
elif dem_type == 'REMA':
    dir_prefix = '/fs/byo/howat-data4' # /fs/project/howat.4 : OLD
else:
    print('Need either ArcticDEM or REMA as input for demtype, EXITING script')
    sys.exit(0)
# strip_version = ['v4', 'unf', 'v4.1']
# strip_version = strip_version[0] #choose based on how folder names look

In [3]:
dir_prefix

'/fs/byo/howat-data5'

In [4]:
strips_folder = f'{dir_prefix}/{dem_type}/{region}/strips_v4.1/2m' #f'{dir_prefix}/EarthDEM/{region}/strips_unf/2m' #for Alaska but not tried yet as ATL06 does not exist for this region
strips = os.listdir(f'{strips_folder}')
strips = [f for f in strips if not f.endswith('.json')]  # Mar 30, 2022 (for new version of DEMs)

logging.info(f'Total Strips for {region}= {len(strips)}')

In [8]:
strips[:10]

['W1W1_20181021_102001007C5E4500_102001007C6AE600_2m_lsf_v040002',
 'W1W1_20181025_1020010078394500_102001007B169300_2m_lsf_v030403',
 'W1W1_20190213_102001007F94EC00_102001008200A500_2m_lsf_v040002',
 'W1W1_20190213_102001007F94EC00_1020010082D88000_2m_lsf_v040002',
 'W1W1_20190213_10200100802C5000_102001008200A500_2m_lsf_v040002',
 'W1W1_20190213_10200100802C5000_1020010083AAFA00_2m_lsf_v040002',
 'W1W1_20190213_1020010083AAFA00_10200100848CC900_2m_lsf_v030403',
 'W1W1_20190403_102001007E600700_1020010083207800_2m_lsf_v040002',
 'W1W1_20190403_1020010081627F00_1020010083207800_2m_lsf_v030403',
 'W1W1_20190403_1020010083207800_1020010085192F00_2m_lsf_v030403']

In [7]:
# Set the icesat2_path : where files will be downloaded and processed (created inside code)
base_icesat2_path = f'/fs/project/howat.4/icesat2/{dem_type}/{region}_{short_name}'
# base_icesat2_path = f'/fs/project/howat.4/icesat2/prototyping/temp/{dem_type}/{region}_{short_name}'

# Get only WV3 files for now
cutoff_dt = pd.to_datetime('2018-10-16') #This is the date when ICESAT started collecting data
strips = [strip for strip in strips if pd.to_datetime(strip.split('_')[1])>=cutoff_dt]

In [9]:
# Aside: next two lines for testing only (March 31, 2022)
strips = [strip for strip in strips if strip.startswith('W3')]
strips = strips[20:120]

logging.info(f'Strips after 2018-10-16 for {region}= {len(strips)}')
# To procwss just one or multiple named strips uncomment this line below
#strips = ['W1W1_20190410_1020010081378B00_10200100850FB100_2m_lsf']
strips

In [11]:
strip = strips[1]
strip

'W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_v040201'

In [12]:
f'{strips_folder}/{strip}'

'/fs/byo/howat-data5/ArcticDEM/arcticdem_02_greenland_southeast/strips_v4.1/2m/W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_v040201'

In [14]:
download_error = []
hdf_error = []

strip_index = 0
strip_order_dict = {} #To save order numbers for each Tile
# for strip in strips:
strip = strips[1]
logging.info('==============================================================================================================')
strip_index += 1 # just to keep count of strips
logging.info(f'{strip_index}/{len(strips)}   {region}:: {strip}')
strip_folder = f'{strips_folder}/{strip}'
icesat2_path = f'{base_icesat2_path}/{strip}'
if os.path.exists(icesat2_path):
    logging.info(f'Skipping Download and Processing {region}: {strip}')
    # TODO: Also check if downloads subfolder exit: this implies 
    print('yes')

In [15]:
# try:
# Download icesat2 data for a REMA strip using API
# Remove trailing _v0xxxx for new ArcticDEM files
#strip = strip.split('_v0')[0]
# In the newer version of ArcticDEM for Greenland, there are some missing strip even though the folder exist
if len(os.listdir(f'{strip_folder}'))<1:
    # Check if no DEM exist; otherwise the script will crash when downloading
    print(f'MISSING STRIP: {strip_folder}')
    logging.error(f'MISSING STRIP: {strip_folder}')
    print('yes')

In [16]:
from icesat2_download import download_process_icesat2

In [23]:
os.listdir(strip_folder)

['SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg1_ortho2.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg1_ortho.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg1_dem.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg2_ortho2.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg2_ortho.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg2_dem.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg1_dem_10m.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg1_dem_10m_shade.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg1_matchtag.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg1_dem_10m_shade_masked.tif',
 'SETSM_s2s041_W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_seg1_bitmask.tif',
 'SETSM_s2s041_

'/fs/byo/howat-data5/ArcticDEM/arcticdem_02_greenland_southeast/strips_v4.1/2m/W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_v040201'

In [18]:
strip

'W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_v040201'

In [22]:
os.listdir(icesat2_path)

FileNotFoundError: [Errno 2] No such file or directory: '/fs/project/howat.4/icesat2/ArcticDEM/arcticdem_02_greenland_southeast_ATL06/W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_v040201'

In [20]:
download_process_icesat2(strip_folder, strip, icesat2_path)

FileNotFoundError: [Errno 2] No such file or directory: '/fs/project/howat.4/icesat2/ArcticDEM/arcticdem_02_greenland_southeast_ATL06/W3W3_20190602_104001004C0F9500_104001004E612800_2m_lsf_v040201'

In [ ]:
orderdict = download_process_icesat2(strip_folder, strip, icesat2_path)
strip_order_dict[strip] = orderdict
# time.sleep(1) #To delay another request to server

In [ ]:
except:
    logging.error(f'Strip_Index: {strip_index}  Exception in data download: {region} {strip}')
    download_error.append(f'{strip}')
#Parsing-----------------------------------------------------
# Think if its better to parse separately
try:
    # Parse HDF file: convert to csv and shapefile/geopackage
    if short_name == 'ATL06':
        read_atl06(icesat2_path)
    if short_name == 'ATL08':
        read_atl08(icesat2_path)
except:
    logging.error(f' Strip_Index: {strip_index}  Exception in processing hdf file: {region} {strip}')
    hdf_error.append(f'{strip}')
logging.info(f'End for strip_index = {strip_index}\n')
#Parsing----------------------------------------------------------------------------------------------------------------------------------        
if len(download_error)>0:
    logging.error('The following list contains Download Errors. Diagnose them separately')
    logging.info(download_error) #save to list of error strips to diagnose later
if len(hdf_error)>0:
    logging.error('The following list contains HDF error. Diagnose them separately')
    logging.info(hdf_error) #save to list of error strips to diagnose later
running_time = toc()
logging.info(f'Total Running Time: {running_time}')    
# Save the orders returned by the nsidc server as csv file
#df = pd.DataFrame.from_dict(strip_order_dict, orient='index')
#df.to_csv(f'{base_icesat2_path}/orders.csv')
print(f"Completed: Search and Download icesat for {short_name}")